# Enhanced Multi-Model Ensemble with Custom Loss Functions

Strategy:
1. Custom asymmetric loss for all three algorithms
2. Advanced feature engineering
3. Validation-based sector analysis
4. Temporal smoothing post-processing
5. Optimized ensemble weights

In [64]:
TEST_MODE = True

## Imports

In [65]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.cluster import KMeans
import optuna  # Bayesian optimization
import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)  # Reduce optuna output

print(f"Running in {'TEST' if TEST_MODE else 'PRODUCTION'} mode")

Running in TEST mode


## Competition Metric + Custom Loss Functions

In [66]:
def competition_metric(y_true, y_pred, eps=1e-10, verbose=True):
    """
    Two-stage metric from competition rules
    """
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    
    ape = np.abs(y_true - y_pred) / np.maximum(y_true, eps)
    
    extreme_errors = (ape > 1.0).sum() / len(ape)
    if verbose:
        print(f"  APE > 100%: {extreme_errors*100:.1f}%")
    
    if extreme_errors > 0.3:
        if verbose:
            print(f"  FAILED Stage 1")
        return 0.0
    
    valid_mask = ape <= 1.0
    if valid_mask.sum() == 0:
        return 0.0
    
    mape = ape[valid_mask].mean()
    fraction_valid = valid_mask.sum() / len(ape)
    scaled_mape = mape / fraction_valid
    score = max(0, 1 - scaled_mape)
    
    if verbose:
        print(f"  Valid samples: {fraction_valid*100:.1f}%")
        print(f"  MAPE (valid): {mape:.4f}")
        print(f"  Scaled MAPE: {scaled_mape:.4f}")
    
    return score


# === ADAPTIVE CATBOOST LOSS ===
class CatBoostAdaptiveLoss(object):
    """Adaptive asymmetric loss - stronger penalty for larger over-predictions"""
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        result = []
        for i in range(len(targets)):
            diff = targets[i] - approxes[i]
            
            # Calculate penalty based on magnitude of over-prediction
            if approxes[i] > targets[i]:
                # Over-predicting
                magnitude = (approxes[i] - targets[i]) / (targets[i] + 1e-10)
                if magnitude > 0.5:  # Extreme over-prediction (>50%)
                    penalty = 10.0
                elif magnitude > 0.2:  # Moderate over-prediction (20-50%)
                    penalty = 5.0
                else:  # Small over-prediction (<20%)
                    penalty = 3.0
                der1 = penalty * np.sign(diff)
            else:
                # Under-predicting - small penalty
                der1 = 1.0 * np.sign(diff)
            
            der2 = 0
            result.append((der1, der2))
        return result


class CatBoostCompetitionMetric:
    def is_max_optimal(self):
        return True
    
    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        score = competition_metric(target, approxes[0], verbose=False)
        return score, 1
    
    def get_final_error(self, error, weight):
        return error


# === ADAPTIVE XGBOOST LOSS ===
def xgb_adaptive_loss(y_pred, dtrain):
    """
    Adaptive asymmetric loss for XGBoost
    Stronger penalty for larger over-predictions
    """
    y_true = dtrain.get_label()
    residual = y_pred - y_true
    
    # Adaptive penalty based on magnitude
    magnitude = np.abs(residual) / (y_true + 1e-10)
    over_pred_mask = y_pred > y_true
    
    # Initialize with base gradient
    grad = np.ones_like(y_true) * residual
    
    # Apply adaptive penalties for over-predictions
    extreme_mask = over_pred_mask & (magnitude > 0.5)
    moderate_mask = over_pred_mask & (magnitude > 0.2) & (magnitude <= 0.5)
    small_mask = over_pred_mask & (magnitude <= 0.2)
    
    grad[extreme_mask] = 10.0 * residual[extreme_mask]
    grad[moderate_mask] = 5.0 * residual[moderate_mask]
    grad[small_mask] = 3.0 * residual[small_mask]
    
    # Hessian (constant for simplicity)
    hess = np.ones_like(y_true)
    
    return grad, hess


# === ADAPTIVE LIGHTGBM LOSS ===
def lgb_adaptive_loss(y_true, y_pred):
    """
    Adaptive asymmetric loss for LightGBM
    """
    residual = y_pred - y_true
    
    # Adaptive penalty
    magnitude = np.abs(residual) / (y_true + 1e-10)
    over_pred_mask = y_pred > y_true
    
    grad = np.ones_like(y_true) * residual
    
    extreme_mask = over_pred_mask & (magnitude > 0.5)
    moderate_mask = over_pred_mask & (magnitude > 0.2) & (magnitude <= 0.5)
    small_mask = over_pred_mask & (magnitude <= 0.2)
    
    grad[extreme_mask] = 10.0 * residual[extreme_mask]
    grad[moderate_mask] = 5.0 * residual[moderate_mask]
    grad[small_mask] = 3.0 * residual[small_mask]
    
    hess = np.ones_like(y_true)
    
    return grad, hess


print("Adaptive loss functions defined")

Adaptive loss functions defined


## Load Data

In [67]:
file_path = "./data/"

new_house = pd.read_csv(file_path + 'train/new_house_transactions.csv')
new_house_nearby = pd.read_csv(file_path + 'train/new_house_transactions_nearby_sectors.csv')
pre_owned = pd.read_csv(file_path + 'train/pre_owned_house_transactions.csv')
pre_owned_nearby = pd.read_csv(file_path + 'train/pre_owned_house_transactions_nearby_sectors.csv')
land_trans = pd.read_csv(file_path + 'train/land_transactions.csv')
land_trans_nearby = pd.read_csv(file_path + 'train/land_transactions_nearby_sectors.csv')
sector_poi = pd.read_csv(file_path + 'train/sector_POI.csv')
city_indexes = pd.read_csv(file_path + 'train/city_indexes.csv')
search_index = pd.read_csv(file_path + 'train/city_search_index.csv')
test = pd.read_csv(file_path + 'test.csv')
test[['month', 'sector']] = test['id'].str.split('_', n=1, expand=True)

print(f"Training samples: {len(new_house)}")
print(f"Test samples: {len(test)}")

Training samples: 5433
Test samples: 1152


## Feature Engineering Functions

In [68]:
def create_base_features(df, target='amount_new_house_transactions', sector_clusters=None):
    """
    Enhanced feature engineering - NO TARGET-DERIVED FEATURES AT ALL
    """
    data = df.copy()
    
    # Merge all datasets EXCEPT new_house (all new_house columns are derived from transactions we're predicting)
    data = data.merge(new_house_nearby, on=['month', 'sector'], how='left')
    data = data.merge(pre_owned, on=['month', 'sector'], how='left')
    data = data.merge(pre_owned_nearby, on=['month', 'sector'], how='left')
    data = data.merge(land_trans, on=['month', 'sector'], how='left')
    data = data.merge(land_trans_nearby, on=['month', 'sector'], how='left')
    data = data.merge(sector_poi, on='sector', how='left')
    
    # DO NOT MERGE new_house AT ALL - every column in it is derived from the target
    # - price_new_house_transactions: calculated FROM actual transactions
    # - num/area_new_house_transactions: the transaction data we're predicting
    # - num/area_new_house_available_for_sale: inventory derived from transaction outcomes
    # - period_new_house_sell_through: calculated FROM transaction velocity
    
    # DateTime features
    data['month_date'] = pd.to_datetime(data['month'], format='%Y-%b' if 'id' not in data.columns else '%Y %b')
    data['year'] = data['month_date'].dt.year
    data['month_num'] = data['month_date'].dt.month
    data['quarter'] = data['month_date'].dt.quarter
    
    # City indexes
    data = data.merge(city_indexes, left_on='year', right_on='city_indicator_data_year', how='left')
    
    # Search aggregates
    search_agg = search_index.groupby('month').agg({
        'search_volume': ['sum', 'mean', 'max', 'std']
    }).reset_index()
    search_agg.columns = ['month', 'search_volume_sum', 'search_volume_mean', 
                          'search_volume_max', 'search_volume_std']
    data = data.merge(search_agg, on='month', how='left')
    
    # Cyclic features
    data['month_sin'] = np.sin(2 * np.pi * data['month_num'] / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['month_num'] / 12)
    data['quarter_sin'] = np.sin(2 * np.pi * data['quarter'] / 4)
    data['quarter_cos'] = np.cos(2 * np.pi * data['quarter'] / 4)
    
    # === RATIO FEATURES (all valid, no target leakage) ===
    
    # Pre-owned market dynamics (separate market from new houses - valid)
    data['nearby_to_local_preowned_ratio'] = (
        data['amount_pre_owned_house_transactions_nearby_sectors'] / 
        (data['amount_pre_owned_house_transactions'] + 1)
    )
    
    # Land transaction features
    data['land_price_per_area'] = (
        data['transaction_amount'] / 
        (data['construction_area'] + 1)
    )
    data['land_planned_vs_construction'] = (
        data['planned_building_area'] / 
        (data['construction_area'] + 1)
    )
    
    # Population-based features
    data['price_per_resident'] = (
        data['surrounding_housing_average_price'] / 
        (data['resident_population'] + 1)
    )
    data['shops_per_population'] = (
        data['number_of_shops'] / 
        (data['resident_population'] + 1)
    )
    data['office_to_resident_ratio'] = (
        data['office_population'] / 
        (data['resident_population'] + 1)
    )
    
    # Sector cluster features (if provided)
    if sector_clusters is not None:
        data = data.merge(sector_clusters, on='sector', how='left')
    
    return data


def create_lag_features(data, target='amount_new_house_transactions'):
    """
    Enhanced lag features - these are VALID (shifted target, no leakage)
    """
    data = data.sort_values(['sector', 'month_date']).copy()
    
    # Target encoding (shifted)
    data['sector_target_encoded'] = data.groupby('sector')[target].transform(
        lambda x: x.shift(1).expanding().mean()
    )
    
    # Expanded lags
    for lag in [1, 2, 3, 4, 5, 6, 7, 9, 12, 18, 24]:
        data[f'amount_lag_{lag}'] = data.groupby('sector')[target].shift(lag)
    
    # EMAs
    for span in [3, 6, 12]:
        data[f'amount_ewm_{span}'] = data.groupby('sector')[target].transform(
            lambda x: x.shift(1).ewm(span=span, adjust=False).mean()
        )
    
    # Rolling stats
    for window in [3, 6, 12]:
        data[f'amount_rolling_mean_{window}'] = data.groupby('sector')[target].transform(
            lambda x: x.shift(1).rolling(window, min_periods=1).mean()
        )
        data[f'amount_rolling_std_{window}'] = data.groupby('sector')[target].transform(
            lambda x: x.shift(1).rolling(window, min_periods=1).std()
        )
    
    # Momentum/acceleration
    data['mom_change_1m'] = (
        (data.groupby('sector')[target].shift(1) - data.groupby('sector')[target].shift(2)) /
        (data.groupby('sector')[target].shift(2) + 1)
    )
    data['mom_change_3m'] = (
        (data.groupby('sector')[target].shift(1) - data.groupby('sector')[target].shift(4)) /
        (data.groupby('sector')[target].shift(4) + 1)
    )
    data['amount_lag_12_growth'] = (
        (data.groupby('sector')[target].shift(1) - data.groupby('sector')[target].shift(13)) /
        (data.groupby('sector')[target].shift(13) + 1)
    )
    data['qoq_growth'] = (
        (data.groupby('sector')[target].shift(1) - data.groupby('sector')[target].shift(4)) /
        (data.groupby('sector')[target].shift(4) + 1)
    )
    data['trend_3m'] = (
        (data.groupby('sector')[target].shift(1) - data.groupby('sector')[target].shift(4)) /
        (data.groupby('sector')[target].shift(4) + 1)
    )
    data['trend_6m'] = (
        (data.groupby('sector')[target].shift(1) - data.groupby('sector')[target].shift(7)) /
        (data.groupby('sector')[target].shift(7) + 1)
    )
    data['acceleration_3m'] = data['trend_3m'] - data.groupby('sector')['trend_3m'].shift(3)
    data['volatility_6m'] = data.groupby('sector')[target].transform(
        lambda x: x.shift(1).rolling(6, min_periods=1).std() / (x.shift(1).rolling(6, min_periods=1).mean() + 1)
    )
    
    return data


def add_sector_stats(data, train_data, target='amount_new_house_transactions'):
    """
    Sector statistics - VALID (from training data only)
    """
    sector_stats = train_data.groupby('sector')[target].agg(['mean', 'std', 'min', 'max', 'median']).reset_index()
    sector_stats.columns = ['sector', 'sector_mean', 'sector_std', 'sector_min', 'sector_max', 'sector_median']
    sector_stats['sector_cv'] = sector_stats['sector_std'] / (sector_stats['sector_mean'] + 1)
    data = data.merge(sector_stats, on='sector', how='left')
    return data


print("Feature engineering functions defined - ZERO TARGET LEAKAGE!")

Feature engineering functions defined - ZERO TARGET LEAKAGE!


## STEP 1: Build Complete Grid + Create Sector Clusters

In [69]:
target = 'amount_new_house_transactions'

# Build dense grid
months = sorted(set(new_house['month']) | set(pre_owned['month']) | set(land_trans['month']))
sectors = sorted(set(new_house['sector']) | set(pre_owned['sector']) | set(land_trans['sector']))
train_grid = pd.DataFrame([(m, s) for m in months for s in sectors], columns=['month', 'sector'])

# Attach target (zeros when missing)
train_grid = train_grid.merge(new_house[['month', 'sector', target]], on=['month', 'sector'], how='left')
train_grid[target] = train_grid[target].fillna(0)

print(f"Grid size: {len(train_grid)} (was {len(new_house)} observed)")
print(f"Zero-filled entries: {(train_grid[target] == 0).sum()}")

# Create sector clusters based on POI features
poi_features = ['resident_population', 'office_population', 'number_of_shops', 
                'transportation_station', 'education', 'surrounding_housing_average_price']
poi_data = sector_poi[['sector'] + poi_features].fillna(0)

# K-Means clustering
n_clusters = 5 if not TEST_MODE else 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
poi_data['sector_cluster'] = kmeans.fit_predict(poi_data[poi_features])
sector_clusters = poi_data[['sector', 'sector_cluster']]

print(f"Created {n_clusters} sector clusters")
print(sector_clusters.groupby('sector_cluster').size())

Grid size: 6432 (was 5433 observed)
Zero-filled entries: 999
Created 3 sector clusters
sector_cluster
0    12
1     1
2    73
dtype: int64


## STEP 2: Feature Engineering + Train/Val Split

In [70]:
# Build features on grid
all_data = create_base_features(train_grid, target=target, sector_clusters=sector_clusters)

# ONLY USE RECENT 24 MONTHS for training (like leader uses 18 months)
# This captures current market regime, not ancient 2019-2020 COVID data
recent_months = 24
cutoff_date = all_data['month_date'].max() - pd.DateOffset(months=recent_months)
all_data = all_data[all_data['month_date'] >= cutoff_date].copy()

print(f"Using only recent {recent_months} months: {all_data['month_date'].min()} to {all_data['month_date'].max()}")

# Time-based split (last 6 months for validation)
val_months = 6
val_cutoff = all_data['month_date'].max() - pd.DateOffset(months=val_months)

train_data = all_data[all_data['month_date'] < val_cutoff].copy()
val_data = all_data[all_data['month_date'] >= val_cutoff].copy()

print(f"Train: {len(train_data)} ({train_data['month_date'].min()} to {train_data['month_date'].max()})")
print(f"Val: {len(val_data)} ({val_data['month_date'].min()} to {val_data['month_date'].max()})")

# Lag features
train_data = create_lag_features(train_data, target=target)

combined = pd.concat([train_data, val_data]).sort_values(['sector', 'month_date'])
combined = create_lag_features(combined, target=target)
val_data = combined[combined['month_date'] >= val_cutoff].copy()

# Sector stats
train_data = add_sector_stats(train_data, train_data, target=target)
val_data = add_sector_stats(val_data, train_data, target=target)

# Define features - REMOVED all new_house inventory columns
exclude_cols = [
    'month', 'sector', 'month_date', target, 'city_indicator_data_year',
]
feature_cols = [col for col in train_data.columns
                if col not in exclude_cols and train_data[col].dtype in ['int64', 'float64', 'int8', 'float32']]

X_train = train_data[feature_cols].fillna(0)
y_train = train_data[target]
X_val = val_data[feature_cols].fillna(0)
y_val = val_data[target]

print(f"Features: {len(feature_cols)}")
print(f"X_train: {X_train.shape}, X_val: {X_val.shape}")

Using only recent 24 months: 2022-07-01 00:00:00 to 2024-07-01 00:00:00
Train: 2304 (2022-07-01 00:00:00 to 2023-12-01 00:00:00)
Val: 672 (2024-01-01 00:00:00 to 2024-07-01 00:00:00)
Features: 289
X_train: (2304, 289), X_val: (672, 289)


## STEP 3: Hyperparameter Tuning (with Custom Loss)

In [71]:
# Choose optimization method - USE OPTUNA IN BOTH MODES
USE_OPTUNA = True  # Always use Bayesian optimization

if TEST_MODE:
    print("[TEST MODE] Using Optuna with reduced trials for quick testing")
    n_trials_xgb = 10  # Quick search
    n_trials_lgb = 10
    n_trials_cat = 15  # More for CatBoost since it's most important
else:
    print("[PRODUCTION MODE] Using Optuna Bayesian optimization")
    n_trials_xgb = 200  # Search 100 combinations intelligently
    n_trials_lgb = 200
    n_trials_cat = 200  # More for CatBoost since it's most important

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# ==============================================================================
# OPTUNA OPTIMIZATION FUNCTIONS
# ==============================================================================

def objective_xgb(trial):
    """Optuna objective for XGBoost"""
    params = {
        'max_depth': trial.suggest_int('max_depth', 5, 10),
        'eta': trial.suggest_float('eta', 0.005, 0.03, log=True),
        'subsample': trial.suggest_float('subsample', 0.7, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.95),
        'gamma': trial.suggest_float('gamma', 0.01, 0.3),
        'alpha': trial.suggest_float('alpha', 0.01, 0.3),
        'lambda': trial.suggest_float('lambda', 0.5, 2.0),
        'seed': 42
    }
    n_est = trial.suggest_int('n_estimators', 1000, 4000, step=500)
    
    model = xgb.train(
        params, dtrain,
        num_boost_round=n_est,
        obj=xgb_adaptive_loss,
        evals=[(dval, 'val')],
        early_stopping_rounds=200 if not TEST_MODE else 50,
        verbose_eval=False
    )
    
    pred = np.maximum(model.predict(dval), 0)
    score = competition_metric(y_val, pred, verbose=False)
    
    # Store best iteration for later use
    trial.set_user_attr('best_iteration', model.best_iteration)
    
    return score


def objective_lgb(trial):
    """Optuna objective for LightGBM"""
    params = {
        'max_depth': trial.suggest_int('max_depth', 5, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 255),
        'subsample': trial.suggest_float('subsample', 0.7, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.95),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 0.3),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.5, 2.0),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 4000, step=500),
        'objective': lgb_adaptive_loss,
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1
    }
    
    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(200 if not TEST_MODE else 50, verbose=False)]
    )
    
    pred = np.maximum(model.predict(X_val), 0)
    score = competition_metric(y_val, pred, verbose=False)
    
    trial.set_user_attr('best_iteration', model.best_iteration_)
    
    return score


def objective_cat(trial):
    """Optuna objective for CatBoost - MOST IMPORTANT"""
    params = {
        'iterations': trial.suggest_int('iterations', 5000, 25000, step=2500) if not TEST_MODE else 500,
        'depth': trial.suggest_int('depth', 5, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.04, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 1.0),
        'random_seed': 42,
        'loss_function': CatBoostAdaptiveLoss(),
        'eval_metric': CatBoostCompetitionMetric(),
        'early_stopping_rounds': 500 if not TEST_MODE else 100,
        'verbose': False
    }
    
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val))
    
    pred = np.maximum(model.predict(X_val), 0)
    score = competition_metric(y_val, pred, verbose=False)
    
    trial.set_user_attr('best_iteration', model.get_best_iteration())
    
    return score


# ==============================================================================
# HYPERPARAMETER OPTIMIZATION WITH OPTUNA
# ==============================================================================

# === XGBOOST OPTIMIZATION ===
print(f"\nOptimizing XGBoost with Optuna ({n_trials_xgb} trials)...")
study_xgb = optuna.create_study(direction='maximize', study_name='xgb_optimization')
study_xgb.optimize(objective_xgb, n_trials=n_trials_xgb, show_progress_bar=True)

best_xgb_params = study_xgb.best_params
best_xgb_params['n_estimators'] = study_xgb.best_trial.user_attrs['best_iteration']
best_xgb_score = study_xgb.best_value

print(f"Best XGBoost params: {best_xgb_params}")
print(f"Best XGBoost score: {best_xgb_score:.4f}")

# === LIGHTGBM OPTIMIZATION ===
print(f"\nOptimizing LightGBM with Optuna ({n_trials_lgb} trials)...")
study_lgb = optuna.create_study(direction='maximize', study_name='lgb_optimization')
study_lgb.optimize(objective_lgb, n_trials=n_trials_lgb, show_progress_bar=True)

best_lgb_params = study_lgb.best_params
best_lgb_params['n_estimators'] = study_lgb.best_trial.user_attrs['best_iteration']
best_lgb_score = study_lgb.best_value

print(f"Best LightGBM params: {best_lgb_params}")
print(f"Best LightGBM score: {best_lgb_score:.4f}")

# === CATBOOST OPTIMIZATION (MOST IMPORTANT!) ===
print(f"\nOptimizing CatBoost with Optuna ({n_trials_cat} trials) - THIS IS KEY...")
study_cat = optuna.create_study(direction='maximize', study_name='cat_optimization')
study_cat.optimize(objective_cat, n_trials=n_trials_cat, show_progress_bar=True)

best_cat_params = study_cat.best_params
best_cat_params['iterations'] = study_cat.best_trial.user_attrs['best_iteration']
best_cat_score = study_cat.best_value

print(f"Best CatBoost params: {best_cat_params}")
print(f"Best CatBoost score: {best_cat_score:.4f}")

# Rename keys to match expected format
if 'eta' in best_xgb_params:
    best_xgb_params['learning_rate'] = best_xgb_params.pop('eta')

print("\n" + "="*60)
print("HYPERPARAMETER SEARCH COMPLETE")
print("="*60)
print(f"Best XGBoost score: {best_xgb_score:.4f}")
print(f"Best LightGBM score: {best_lgb_score:.4f}")
print(f"Best CatBoost score: {best_cat_score:.4f}")
print("="*60)

[TEST MODE] Using Optuna with reduced trials for quick testing

Optimizing XGBoost with Optuna (10 trials)...


Best trial: 9. Best value: 0.505315: 100%|██████████| 10/10 [00:52<00:00,  5.27s/it]


Best XGBoost params: {'max_depth': 5, 'eta': 0.00597986549809105, 'subsample': 0.7075688195363431, 'colsample_bytree': 0.8595248529280993, 'gamma': 0.1670166652998772, 'alpha': 0.1860811063711681, 'lambda': 1.8621618644332167, 'n_estimators': 665}
Best XGBoost score: 0.5053

Optimizing LightGBM with Optuna (10 trials)...


Best trial: 4. Best value: 0.517638: 100%|██████████| 10/10 [00:11<00:00,  1.19s/it]


Best LightGBM params: {'max_depth': 10, 'learning_rate': 0.013790438408195164, 'num_leaves': 118, 'subsample': 0.8487802322757152, 'colsample_bytree': 0.8384259258595648, 'reg_alpha': 0.1538328052586716, 'reg_lambda': 0.5506559891687581, 'n_estimators': 147}
Best LightGBM score: 0.5176

Optimizing CatBoost with Optuna (15 trials) - THIS IS KEY...


Best trial: 4. Best value: 0.456216: 100%|██████████| 15/15 [01:00<00:00,  4.05s/it]

Best CatBoost params: {'depth': 5, 'learning_rate': 0.022658164793916934, 'l2_leaf_reg': 0.23438768621794598, 'iterations': 499}
Best CatBoost score: 0.4562

HYPERPARAMETER SEARCH COMPLETE
Best XGBoost score: 0.5053
Best LightGBM score: 0.5176
Best CatBoost score: 0.4562


## STEP 4: Train Ensemble with Custom Loss

In [72]:
seeds = [42, 123] if TEST_MODE else [42, 123, 456, 789, 2024, 1337, 9999, 777, 888, 111, 222, 333, 444, 555, 666]

print(f"\nTraining {len(seeds)} models per algorithm with adaptive loss...")

# XGBoost (native API with adaptive loss)
xgb_models = []
for seed in seeds:
    # Handle Optuna params (has 'eta') vs grid search params (has 'learning_rate')
    lr = best_xgb_params.get('eta', best_xgb_params.get('learning_rate', 0.02))
    
    params = {
        'max_depth': best_xgb_params['max_depth'],
        'eta': lr,
        'subsample': best_xgb_params.get('subsample', 0.8),
        'colsample_bytree': best_xgb_params.get('colsample_bytree', 0.8),
        'gamma': best_xgb_params.get('gamma', 0.1),
        'alpha': best_xgb_params.get('alpha', 0.1),
        'lambda': best_xgb_params.get('lambda', 1.0),
        'seed': seed
    }
    model = xgb.train(
        params, dtrain,
        num_boost_round=best_xgb_params['n_estimators'],
        obj=xgb_adaptive_loss,
        verbose_eval=False
    )
    xgb_models.append(model)
print(f"  XGBoost: {len(xgb_models)} models trained")

# LightGBM (sklearn API with adaptive loss)
lgb_models = []
for seed in seeds:
    model = lgb.LGBMRegressor(
        n_estimators=best_lgb_params['n_estimators'],
        max_depth=best_lgb_params['max_depth'],
        learning_rate=best_lgb_params['learning_rate'],
        num_leaves=best_lgb_params.get('num_leaves', 2**best_lgb_params['max_depth']),
        subsample=best_lgb_params.get('subsample', 0.8),
        colsample_bytree=best_lgb_params.get('colsample_bytree', 0.8),
        reg_alpha=best_lgb_params.get('reg_alpha', 0.1),
        reg_lambda=best_lgb_params.get('reg_lambda', 1.0),
        objective=lgb_adaptive_loss,
        random_state=seed,
        n_jobs=-1,
        verbose=-1
    )
    model.fit(X_train, y_train)
    lgb_models.append(model)
print(f"  LightGBM: {len(lgb_models)} models trained")

# CatBoost (with adaptive loss)
cat_models = []
for seed in seeds:
    model = CatBoostRegressor(
        iterations=best_cat_params['iterations'],
        depth=best_cat_params['depth'],
        learning_rate=best_cat_params['learning_rate'],
        l2_leaf_reg=best_cat_params.get('l2_leaf_reg', 0.3),
        random_seed=seed,
        loss_function=CatBoostAdaptiveLoss(),
        eval_metric=CatBoostCompetitionMetric(),
        metric_period=9999,
        verbose=False
    )
    model.fit(X_train, y_train, eval_set=(X_val, y_val))
    cat_models.append(model)
print(f"  CatBoost: {len(cat_models)} models trained")


Training 2 models per algorithm with adaptive loss...
  XGBoost: 2 models trained
  LightGBM: 2 models trained
  CatBoost: 2 models trained


## STEP 5: Optimize Ensemble Weights

In [73]:
# Get predictions (ensure non-negative)
xgb_val_preds = np.maximum(np.mean([m.predict(dval) for m in xgb_models], axis=0), 0)
lgb_val_preds = np.maximum(np.mean([m.predict(X_val) for m in lgb_models], axis=0), 0)
cat_val_preds = np.maximum(np.mean([m.predict(X_val) for m in cat_models], axis=0), 0)

print("Optimizing ensemble weights...")

grid = np.linspace(0, 1, 11 if TEST_MODE else 21)
best_score = -1
best_weights = None

for w_xgb in grid:
    for w_lgb in grid:
        w_cat = 1.0 - w_xgb - w_lgb
        if w_cat < 0 or w_cat > 1:
            continue
        
        ensemble_pred = w_xgb * xgb_val_preds + w_lgb * lgb_val_preds + w_cat * cat_val_preds
        score = competition_metric(y_val, ensemble_pred, verbose=False)
        
        if score > best_score:
            best_score = score
            best_weights = (w_xgb, w_lgb, w_cat)
            print(f"  New best: XGB={w_xgb:.2f}, LGB={w_lgb:.2f}, CAT={w_cat:.2f}, score={score:.4f}")

print(f"\n=== Best Ensemble ===")
print(f"XGBoost:  {best_weights[0]:.3f}")
print(f"LightGBM: {best_weights[1]:.3f}")
print(f"CatBoost: {best_weights[2]:.3f}")
print(f"Val Score: {best_score:.4f}")

final_val_pred = (
    best_weights[0] * xgb_val_preds + 
    best_weights[1] * lgb_val_preds + 
    best_weights[2] * cat_val_preds
)

print("\n=== Validation Performance ===")
competition_metric(y_val, final_val_pred)
print(f"MAE: {mean_absolute_error(y_val, final_val_pred):.2f}")

Optimizing ensemble weights...
  New best: XGB=0.00, LGB=0.00, CAT=1.00, score=0.4511
  New best: XGB=0.00, LGB=0.30, CAT=0.70, score=0.4697
  New best: XGB=0.00, LGB=0.40, CAT=0.60, score=0.4885
  New best: XGB=0.00, LGB=0.50, CAT=0.50, score=0.5043
  New best: XGB=0.00, LGB=0.60, CAT=0.40, score=0.5154
  New best: XGB=0.00, LGB=0.70, CAT=0.30, score=0.5235

=== Best Ensemble ===
XGBoost:  0.000
LightGBM: 0.700
CatBoost: 0.300
Val Score: 0.5235

=== Validation Performance ===
  APE > 100%: 23.2%
  Valid samples: 76.8%
  MAPE (valid): 0.3659
  Scaled MAPE: 0.4765
MAE: 10235.80


## STEP 6: Validation-Based Sector Analysis

In [74]:
# Analyze which sectors should be zeroed based on validation errors
val_analysis = val_data[['sector', target]].copy()
val_analysis['prediction'] = final_val_pred
val_analysis['ape'] = np.abs(val_analysis[target] - val_analysis['prediction']) / np.maximum(val_analysis[target], 1e-10)

# Identify sectors with consistently high errors OR consistently zero transactions
sector_error_analysis = val_analysis.groupby('sector').agg({
    target: ['mean', 'max', 'sum'],
    'ape': 'mean',
    'prediction': 'mean'
}).reset_index()
sector_error_analysis.columns = ['sector', 'target_mean', 'target_max', 'target_sum', 'ape_mean', 'pred_mean']

# Sectors to zero:
# 1. All transactions in val are zero
# 2. OR mean APE > 2.0 (very bad predictions)
zero_sectors = sector_error_analysis[
    (sector_error_analysis['target_sum'] == 0) | 
    (sector_error_analysis['ape_mean'] > 2.0)
]['sector'].tolist()

print(f"\nSectors to zero (validation-based): {len(zero_sectors)}")
print(f"Sectors: {sorted(zero_sectors)}")


Sectors to zero (validation-based): 24
Sectors: ['sector 12', 'sector 13', 'sector 17', 'sector 19', 'sector 26', 'sector 3', 'sector 33', 'sector 39', 'sector 40', 'sector 41', 'sector 42', 'sector 44', 'sector 49', 'sector 52', 'sector 53', 'sector 72', 'sector 73', 'sector 74', 'sector 75', 'sector 82', 'sector 87', 'sector 89', 'sector 95', 'sector 96']


## STEP 7: Retrain on Full Data

In [75]:
print("=== RETRAINING ON FULL DATA ===")

# Rebuild grid
full_grid = pd.DataFrame([(m, s) for m in months for s in sectors], columns=['month', 'sector'])
full_grid = full_grid.merge(new_house[['month', 'sector', target]], on=['month', 'sector'], how='left')
full_grid[target] = full_grid[target].fillna(0)

full_data = create_base_features(full_grid, target=target, sector_clusters=sector_clusters)
full_data = create_lag_features(full_data, target=target)
full_data = add_sector_stats(full_data, full_data, target=target)

X_full = full_data[feature_cols].fillna(0)
y_full = full_data[target]

print(f"Full training set: {X_full.shape}")
print(f"Features: {len(feature_cols)}")

# Calculate caps
hist_q99 = y_full.quantile(0.99)
last_train_month = full_data['month_date'].max()
window_start = last_train_month - pd.DateOffset(months=6)
recent = full_data[full_data['month_date'] >= window_start]
sector_recent_max = recent.groupby('sector')[target].max()
sector_all_max = full_data.groupby('sector')[target].max()
sector_cap_series = (1.2 * sector_recent_max).fillna(1.2 * sector_all_max)
sector_caps = sector_cap_series.to_dict()
global_cap = float(hist_q99 * 1.5)

print(f"Global cap: {global_cap:.2f}")

# Create DMatrix for full data (for XGBoost)
dfull = xgb.DMatrix(X_full, label=y_full)

# Train final models
print("\nTraining final XGBoost...")
final_xgb_models = []
for seed in seeds:
    params = {
        'max_depth': best_xgb_params['max_depth'],
        'eta': best_xgb_params['learning_rate'],
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'gamma': 0.1,
        'alpha': 0.1,
        'lambda': 1.0,
        'seed': seed
    }
    model = xgb.train(params, dfull, num_boost_round=best_xgb_params['n_estimators'],
                     obj=xgb_adaptive_loss, verbose_eval=False)
    final_xgb_models.append(model)
print(f"  {len(final_xgb_models)} models ✓")

print("\nTraining final LightGBM...")
final_lgb_models = []
for seed in seeds:
    model = lgb.LGBMRegressor(
        n_estimators=best_lgb_params['n_estimators'],
        max_depth=best_lgb_params['max_depth'],
        learning_rate=best_lgb_params['learning_rate'],
        num_leaves=2**best_lgb_params['max_depth'],
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        objective=lgb_adaptive_loss,
        random_state=seed,
        n_jobs=-1,
        verbose=-1
    )
    model.fit(X_full, y_full)
    final_lgb_models.append(model)
print(f"  {len(final_lgb_models)} models ✓")

print("\nTraining final CatBoost...")
final_cat_models = []
for seed in seeds:
    model = CatBoostRegressor(
        iterations=best_cat_params['iterations'],
        depth=best_cat_params['depth'],
        learning_rate=best_cat_params['learning_rate'],
        l2_leaf_reg=0.3,
        random_seed=seed,
        loss_function=CatBoostAdaptiveLoss(),
        eval_metric='MAE',  # Use standard metric for final training
        verbose=False
    )
    model.fit(X_full, y_full)
    final_cat_models.append(model)
print(f"  {len(final_cat_models)} models ✓")

=== RETRAINING ON FULL DATA ===
Full training set: (7584, 289)
Features: 289
Global cap: 336131.37

Training final XGBoost...
  2 models ✓

Training final LightGBM...
  2 models ✓

Training final CatBoost...
  2 models ✓


In [76]:
## ITERATIVE FORECASTING - Predict one month at a time, adding predictions to lag features

print("="*80)
print("ITERATIVE FORECASTING - Predicting 12 months sequentially")
print("="*80)

# Start with full training data up to Jul 2024
forecast_data = full_data.copy()

# Test months: Aug 2024 through Jul 2025
test_months_list = pd.date_range(start='2024-08-01', end='2025-07-01', freq='MS')

all_predictions = []

for i, forecast_month in enumerate(test_months_list, 1):
    forecast_month_str = forecast_month.strftime('%Y %b')
    print(f"\n[{i}/12] Predicting {forecast_month_str}...")
    
    # Get sectors to predict for this month
    month_test = test[test['month'] == forecast_month_str].copy()
    
    # Create features for this month
    month_test_data = create_base_features(month_test, target=target, sector_clusters=sector_clusters)
    
    # Build lag features using ACTUAL historical data + any previous predictions
    combined_for_lags = pd.concat([
        forecast_data[['month', 'sector', 'month_date', target]],
        month_test_data[['month', 'sector', 'month_date']].assign(**{target: np.nan})
    ]).sort_values(['sector', 'month_date']).reset_index(drop=True)
    
    combined_for_lags = create_lag_features(combined_for_lags, target=target)
    month_features = combined_for_lags[combined_for_lags[target].isna()].reset_index(drop=True)
    
    # Transfer lag features
    lag_cols = ['sector_target_encoded'] + \
               [f'amount_lag_{lag}' for lag in [1,2,3,4,5,6,7,9,12,18,24]] + \
               [f'amount_ewm_{span}' for span in [3,6,12]] + \
               [f'amount_rolling_mean_{w}' for w in [3,6,12]] + \
               [f'amount_rolling_std_{w}' for w in [3,6,12]] + \
               ['amount_lag_12_growth', 'trend_3m', 'trend_6m', 'volatility_6m',
                'mom_change_1m', 'mom_change_3m', 'qoq_growth', 'acceleration_3m']
    
    for col in lag_cols:
        month_test_data[col] = month_features[col].values
    
    # Add sector stats
    month_test_data = add_sector_stats(month_test_data, forecast_data, target=target)
    
    # Prepare features
    X_month = month_test_data[feature_cols].fillna(0)
    
    # Make predictions with ensemble
    dmonth = xgb.DMatrix(X_month)
    xgb_preds = np.maximum(np.mean([m.predict(dmonth) for m in final_xgb_models], axis=0), 0)
    lgb_preds = np.maximum(np.mean([m.predict(X_month) for m in final_lgb_models], axis=0), 0)
    cat_preds = np.maximum(np.mean([m.predict(X_month) for m in final_cat_models], axis=0), 0)
    
    month_pred = best_weights[0] * xgb_preds + best_weights[1] * lgb_preds + best_weights[2] * cat_preds
    
    # Post-processing for this month
    # 1. Sector-wise caps
    caps = np.array([sector_caps.get(s, global_cap) for s in month_test['sector']])
    month_pred = np.minimum(month_pred, caps)
    
    # 2. Zero-history heuristic
    lag_check_cols = [f'amount_lag_{k}' for k in [1,2,3,6]]
    zero_hist_mask = month_test_data[lag_check_cols].fillna(0).sum(axis=1) == 0
    month_pred[zero_hist_mask.values] = 0
    
    # 3. Manual sector zeroing (ONLY leader's list, NOT validation-based)
    leader_zero_sectors = [11, 38, 40, 43, 48, 51, 52, 57, 71, 72, 73, 74, 81, 86, 88, 94, 95]
    for sector_num in leader_zero_sectors:
        sector_name = f'sector {sector_num}'
        month_pred[month_test['sector'] == sector_name] = 0
    
    # REMOVED: Validation-based zeroing (was zeroing too many sectors)
    # REMOVED: Uncertainty decay (was killing predictions for later months)
    
    print(f"  Predictions: mean={month_pred.mean():.2f}, median={np.median(month_pred):.2f}, zeros={(month_pred==0).sum()}/{len(month_pred)}")
    
    # Store predictions
    month_results = month_test[['id', 'month', 'sector']].copy()
    month_results['prediction'] = month_pred
    all_predictions.append(month_results)
    
    # CRITICAL: Add predictions to forecast_data for next iteration
    # This creates a feedback loop where future predictions use past predictions as features
    new_rows = month_test_data[['month', 'sector', 'month_date']].copy()
    new_rows[target] = month_pred
    
    # Add all feature columns needed
    for col in forecast_data.columns:
        if col not in new_rows.columns:
            if col in month_test_data.columns:
                new_rows[col] = month_test_data[col].values
            else:
                new_rows[col] = np.nan
    
    forecast_data = pd.concat([forecast_data, new_rows], ignore_index=True)
    forecast_data = forecast_data.sort_values(['sector', 'month_date']).reset_index(drop=True)

# Combine all monthly predictions
final_predictions = pd.concat(all_predictions, ignore_index=True)
final_predictions = final_predictions.sort_values('id').reset_index(drop=True)

print(f"\n{'='*80}")
print(f"ITERATIVE FORECASTING COMPLETE")
print(f"{'='*80}")
print(f"Total predictions: {len(final_predictions)}")
print(f"Predicted range: {final_predictions['prediction'].min():.2f} to {final_predictions['prediction'].max():.2f}")
print(f"Predicted mean: {final_predictions['prediction'].mean():.2f}")
print(f"Zeros: {(final_predictions['prediction'] == 0).sum()} / {len(final_predictions)} ({(final_predictions['prediction'] == 0).sum()/len(final_predictions)*100:.1f}%)")

ITERATIVE FORECASTING - Predicting 12 months sequentially

[1/12] Predicting 2024 Aug...
  Predictions: mean=8969.59, median=4176.58, zeros=22/96

[2/12] Predicting 2024 Sep...
  Predictions: mean=6990.21, median=3905.32, zeros=23/96

[3/12] Predicting 2024 Oct...
  Predictions: mean=5576.52, median=3331.70, zeros=22/96

[4/12] Predicting 2024 Nov...
  Predictions: mean=5012.90, median=2674.55, zeros=24/96

[5/12] Predicting 2024 Dec...
  Predictions: mean=6140.97, median=3666.70, zeros=23/96

[6/12] Predicting 2025 Jan...
  Predictions: mean=4628.27, median=2506.69, zeros=24/96

[7/12] Predicting 2025 Feb...
  Predictions: mean=3454.06, median=1610.67, zeros=35/96

[8/12] Predicting 2025 Mar...
  Predictions: mean=3362.90, median=858.80, zeros=34/96

[9/12] Predicting 2025 Apr...
  Predictions: mean=3037.49, median=873.54, zeros=33/96

[10/12] Predicting 2025 May...
  Predictions: mean=2781.83, median=664.44, zeros=36/96

[11/12] Predicting 2025 Jun...
  Predictions: mean=2651.05, med

## STEP 8: Iterative Forecasting (1-month ahead, 12 iterations)

## STEP 9: Prepare Final Submission

In [77]:
# No temporal smoothing needed - iterative forecasting already accounts for temporal dependencies

test_pred = final_predictions['prediction'].values

print(f"\nFinal Predictions (from iterative forecasting):")
print(f"  Range: {test_pred.min():.2f} to {test_pred.max():.2f}")
print(f"  Mean: {test_pred.mean():.2f}")
print(f"  Median: {np.median(test_pred):.2f}")
print(f"  Zeros: {(test_pred == 0).sum()} / {len(test_pred)} ({(test_pred == 0).sum()/len(test_pred)*100:.1f}%)")


Final Predictions (from iterative forecasting):
  Range: 0.00 to 55662.66
  Mean: 4598.19
  Median: 1989.85
  Zeros: 346 / 1152 (30.0%)


In [78]:
submission = pd.DataFrame({
    'id': test['id'],
    'new_house_transaction_amount': test_pred
})

orig_test = pd.read_csv(file_path + 'test.csv')
assert submission['id'].tolist() == orig_test['id'].tolist(), "ID order mismatch!"
assert np.isfinite(submission['new_house_transaction_amount']).all(), "Non-finite predictions!"

submission.to_csv('submission.csv', index=False)
print("✅ Submission saved to submission.csv")

print("\nFirst 10:")
print(submission.head(10))
print("\nLast 10:")
print(submission.tail(10))

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Mode: {'TEST' if TEST_MODE else 'PRODUCTION'}")
print(f"Best XGBoost: {best_xgb_params}, score: {best_xgb_score:.4f}")
print(f"Best LightGBM: {best_lgb_params}, score: {best_lgb_score:.4f}")
print(f"Best CatBoost: {best_cat_params}, score: {best_cat_score:.4f}")
print(f"Ensemble weights: XGB={best_weights[0]:.3f}, LGB={best_weights[1]:.3f}, CAT={best_weights[2]:.3f}")
print(f"Validation score: {best_score:.4f}")
print(f"Sectors auto-zeroed: {len(zero_sectors)}")
print("="*60)

✅ Submission saved to submission.csv

First 10:
                   id  new_house_transaction_amount
0   2024 Aug_sector 1                   6063.068469
1   2024 Aug_sector 2                  17055.620439
2   2024 Aug_sector 3                      0.000000
3   2024 Aug_sector 4                    565.776000
4   2024 Aug_sector 5                   3352.693035
5   2024 Aug_sector 6                  16464.841388
6   2024 Aug_sector 7                  15837.944512
7   2024 Aug_sector 8                  19115.401146
8   2024 Aug_sector 9                   3837.228000
9  2024 Aug_sector 10                  16447.806999

Last 10:
                      id  new_house_transaction_amount
1142  2025 Jul_sector 87                      0.000000
1143  2025 Jul_sector 88                    655.793147
1144  2025 Jul_sector 89                   2577.240913
1145  2025 Jul_sector 90                     41.020657
1146  2025 Jul_sector 91                   3523.717828
1147  2025 Jul_sector 92                